##### Copyright 2019 The TensorFlow Authors.

# Get started with TensorBoard


기계 학습에서 무언가를 향상시키기 위해서는 종종 그것을 측정 할 수 있어야합니다. TensorBoard는 머신 러닝 워크 플로우 중에 필요한 측정 및 시각화를 제공하는 도구입니다. 손실 및 정확도와 같은 실험 지표를 추적하고, 모델 그래프를 시각화하고, 더 낮은 차원 공간에 임베딩을 투영하는 등 다양한 기능을 제공합니다.

이 빠른 시작에서는 TensorBoard를 빠르게 시작하는 방법을 보여줍니다. 이 웹 사이트의 나머지 안내서는 특정 기능에 대한 자세한 내용을 제공하며, 여기에 포함되지 않은 기능이 많습니다.

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import tensorflow as tf
import datetime

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/ 



[MNIST](https://en.wikipedia.org/wiki/MNIST_database) 데이터 세트를 예로 사용하여 데이터를 정규화하고 이미지를 10 개의 클래스로 분류하기위한 간단한 Keras 모델을 작성하는 함수를 작성하세요.

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

## Using TensorBoard with Keras Model.fit()

Keras의 [Model.fit()](https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#fit)을 사용하여 훈련 할 때 `tf.keras.callbacks.TensorBoard`  콜백을 추가하면 로그가 생성되고 저장됩니다. 또한 `histogram_freq=1`을 사용하여 에포크마다 히스토그램 계산을 활성화합니다 (기본적으로 꺼져 있음).

다양한 훈련 실행을 쉽게 선택할 수 있도록 타임 스탬프가 지정된 하위 디렉토리에 로그를 배치하세요.

In [ ]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

명령 행 또는 노트북 환경에서 TensorBoard를 시작하세요. 두 인터페이스는 일반적으로 동일합니다. 노트북에서는 `% tensorboard` line magic을 사용하세요. 명령 행에서 "%"없이 동일한 명령을 실행하세요.

In [ ]:
%tensorboard --logdir logs/fit

<img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/quickstart_model_fit.png?raw=1"/>

표시된 대시 보드에 대한 간략한 개요는 아래와 같습니다.

* **Scalars** 대시 보드는 손실과 지표가 모든 시대에 어떻게 변하는 지 보여줍니다. 이를 사용하여 훈련 속도, 학습 속도 및 기타 스칼라 값을 추적 할 수도 있습니다.

* **Graphs** 대시 보드는 모델을 시각화하는 데 도움이됩니다. 이 경우 레이어의 Keras 그래프가 표시되어 레이어가 올바르게 작성되었는지 확인할 수 있습니다.

* **Distributions** 및 **Histograms** 대시 보드에는 시간 경과에 따른 텐서 분포가 표시됩니다. 이것은 가중치와 바이어스를 시각화하고 예상 방식으로 변경되는지 확인하는 데 유용 할 수 있습니다.

다른 유형의 데이터를 기록하면 추가 TensorBoard 플러그인이 자동으로 활성화됩니다. 예를 들어 Keras TensorBoard 콜백을 사용하면 이미지와 임베드도 기록 할 수 있습니다. 오른쪽 상단의 "inactive"드롭 다운을 클릭하여 TensorBoard에서 사용 가능한 다른 플러그인을 확인할 수 있습니다.

## Using TensorBoard with other methods

[`tf.GradientTape ()`](https://www.tensorflow.org/api_docs/python/tf/GradientTape)와 같은 방법으로 훈련 할 때는`tf.summary`를 사용하여 필요한 정보를 기록하세요.

위와 동일한 데이터 세트를 사용하지만 배치 기능을 활용하려면`tf.data.Dataset`으로 변환합니다.

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

교육 코드는 [advanced quickstart](https://www.tensorflow.org/tutorials/quickstart/advanced) 자습서를 따르지만 TensorBoard에 메트릭을 기록하는 방법을 보여줍니다. 손실 및 최적화 도구를 선택하세요.

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()


학습 중에 값을 누적하고 어떤 포인트에서든 값을 저장할 수 있는 메트릭을 작성합니다.

In [ ]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

학습 및 평가 함수를 작성합니다.

In [ ]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

Summary Writer를 설정하여 다른 로그 디렉토리의 디스크에 요약을 작성합니다.

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

훈련을 시작하세요.

요약 작성기의 범위 내에서 훈련 / 테스트하는 동안 `tf.summary.scalar()`를 사용하여 메트릭 (손실 및 정확성)을 기록하여 그 요약을 디스크에 작성합니다. 기록 할 메트릭과 빈도를 제어 할 수 있습니다. 다른 `tf.summary()` 함수는 다른 유형의 데이터 로깅을 가능하게합니다.

In [ ]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

TensorBoard를 다시 열고 이번에는 새 로그 디렉토리를 지정합니다. 훈련이 진행되는 동안 모니터하기 위해 TensorBoard를 시작할수도 있었습니다.

In [ ]:
%tensorboard --logdir logs/gradient_tape

<img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/quickstart_gradient_tape.png?raw=1"/>

그게 다야! 이제 더 많은 사용자 정의 시나리오를 위해 Keras 콜백과 `tf.summary`를 통해 TensorBoard를 사용하는 방법을 보았습니다.